# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
print(len(city_data_df))

# Display sample data
city_data_df.head()

583


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,stanley,54.8680,-1.6985,14.48,82,20,3.09,GB,1693253478
1,1,saudarkrokur,65.7461,-19.6394,9.02,94,79,3.02,IS,1693253479
2,2,hamilton,39.1834,-84.5333,28.61,50,20,3.60,US,1693253479
3,3,grytviken,-54.2811,-36.5092,-3.65,80,100,2.33,GS,1693253480
4,4,port-aux-francais,-49.3500,70.2167,3.26,79,82,17.53,TF,1693253480


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

12


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,iqaluit,63.7506,-68.5145,7.85,61,75,5.66,CA,1693253483
19,19,edinburgh of the seven seas,-37.0676,-12.3116,11.86,66,10,6.86,SH,1693253488
47,47,quellon,-43.1167,-73.6167,10.28,63,75,2.12,CL,1693253500
110,110,shahrak,34.1074,64.3052,10.57,49,0,1.65,AF,1693253532
120,120,dunedin,-45.8742,170.5036,10.45,66,52,4.92,NZ,1693253542


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
10,iqaluit,CA,63.7506,-68.5145,61,
19,edinburgh of the seven seas,SH,-37.0676,-12.3116,66,
47,quellon,CL,-43.1167,-73.6167,63,
110,shahrak,AF,34.1074,64.3052,49,
120,dunedin,NZ,-45.8742,170.5036,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#         # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df.head()

Starting hotel search
iqaluit - nearest hotel: Frobisher Inn
edinburgh of the seven seas - nearest hotel: No hotel found
quellon - nearest hotel: Aliwen
shahrak - nearest hotel: No hotel found
dunedin - nearest hotel: Scenic Hotel Dunedin City
qaqortoq - nearest hotel: Hotel Qaqortoq
punta arenas - nearest hotel: Hotel Lacolet
chonchi - nearest hotel: Hotel Huildín
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
puerto deseado - nearest hotel: Los Olmos
ancud - nearest hotel: Cabañas
osorno - nearest hotel: Hotel Sonesta Osorno


,City,Country,Lat,Lng,Humidity,Hotel Name
10,iqaluit,CA,63.7506,-68.5145,61,Frobisher Inn
19,edinburgh of the seven seas,SH,-37.0676,-12.3116,66,No hotel found
47,quellon,CL,-43.1167,-73.6167,63,Aliwen
110,shahrak,AF,34.1074,64.3052,49,No hotel found
120,dunedin,NZ,-45.8742,170.5036,66,Scenic Hotel Dunedin City


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_4

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)